In [15]:
import numpy as np

# Hyperparameters
hidden_size = 100
seq_length = 5
learning_rate = 1e-1

# Data
data = "simargill"
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

# Model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = np.zeros((hidden_size, 1))                        # hidden bias
by = np.zeros((vocab_size, 1))                         # output bias


In [16]:
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    # Forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1))   # one-hot
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # softmax
        loss += -np.log(ps[t][targets[t], 0])         # cross-entropy loss

    # Backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # tanh'
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


In [17]:
n, p = 0, 0
hprev = np.zeros((hidden_size,1))

while n <= 1000:
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN
        p = 0

    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)

    # Update weights with Adagrad
    for param, dparam in zip([Wxh, Whh, Why, bh, by], 
                             [dWxh, dWhh, dWhy, dbh, dby]):
        param -= learning_rate * dparam

    if n % 100 == 0:
        print(f"Iter {n}, Loss: {loss:.4f}")
    
    p += seq_length
    n += 1


Iter 0, Loss: 9.7284
Iter 100, Loss: 0.0188
Iter 200, Loss: 0.0068
Iter 300, Loss: 0.0040
Iter 400, Loss: 0.0028
Iter 500, Loss: 0.0022
Iter 600, Loss: 0.0018
Iter 700, Loss: 0.0015
Iter 800, Loss: 0.0013
Iter 900, Loss: 0.0011
Iter 1000, Loss: 0.0010


In [18]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ''.join(ix_to_char[ix] for ix in ixes)


In [20]:
# After training loop
print("\nSample generated text:")
sampled_text = sample(hprev, seed_ix=char_to_ix['i'], n=100)
print(sampled_text)



Sample generated text:
imargimargimargimargimargimargimargimargimargimargimargimargimargimargimargimargimargimargimargimarg
